#### Load modules

In [18]:
import pandas as pd
import sys, os
from gpt4all import GPT4All
from tqdm import tqdm
import re
from deep_translator import GoogleTranslator

sys.path.append("\\".join(os.getcwd().split("\\")[:-1]))
%load_ext autoreload
%autoreload 2
from utils import *
from scraper import utils

c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\torch\__init__.py:749: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:433.)
  _C._set_default_tensor_type(t)


#### Import data and language model

In [2]:
data = pd.read_pickle(os.path.join("/".join(os.getcwd().split("\\")[:-1]),
                                   'scraper','data/merged_data.pkl'))
# define the language model (Llama with 8 billion parameters is about 4.66 GB)
model = GPT4All('Meta-Llama-3-8B-Instruct.Q4_0.gguf')

In [12]:
print(data.loc[data['title'].str.contains('Feuerwehrplan')]['title_it'].to_markdown())

|       | title_it                       |
|------:|:-------------------------------|
|    32 | Piano dei vigili del fuoco Gde |
|   231 | Piano dei vigili del fuoco Gde |
| 18850 | Piano dei vigili del fuoco Gde |
| 19052 | Piano dei vigili del fuoco Gde |


In [4]:
l = "casa"
l = l.replace("casa","")
len(l.split())

0

#### Clean the input database

In [4]:
language_dict = {'english':('EN', 'ENG'), 'french':('FR','FRA'), 'german':('DE','DEU'), 'italian':('IT','ITA'), 'not_found':('NA','NAN')}
data['lang_3'] = data.apply(lambda row: language_dict[detect_language(row['abstract'], not_found=True)][1], axis=1)
data['lang_2'] = data.apply(lambda row: language_dict[detect_language(row['abstract'], not_found=True)][0], axis=1)
# for idx, row in data.iterrows():
#     if row['abstract'] != 'nan':
#         if language_dict[detect_language(row['abstract_it'])][1] != 'ITA':
#             trnd = GoogleTranslator(source='auto', target='ita').translate(row['abstract'])
#             data.loc[idx, 'abstract_it'] = trnd
#         if language_dict[detect_language(row['abstract_de'])][1] != 'DEU':
#             trnd = GoogleTranslator(source='auto', target='de').translate(row['abstract'])
#             data.loc[idx, 'abstract_de'] = trnd
#         if language_dict[detect_language(row['abstract_fr'])][1] != 'FRA':
#             trnd = GoogleTranslator(source='auto', target='fr').translate(row['abstract'])
#             data.loc[idx, 'abstract_fr'] = trnd
#         if language_dict[detect_language(row['abstract_en'])][1] != 'ENG':
#             trnd = GoogleTranslator(source='auto', target='en').translate(row['abstract'])
#             data.loc[idx, 'abstract_en'] = trnd

c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for 

#### Extract keywords with a local Llama model

In [13]:
data['abstract_w_count'] = data.apply(lambda x: len(x['abstract'].split(' ')), axis=1)
data.apply(lambda x: 'nan' if x['abstract'].startswith(('??','Es werden die Daten im Zeitraum','Link zu Metadaten:',
                                                'https:',
                                                'geo@bs.ch',
                                                'info.geoportal@be.ch',
                                                'sit@jura.ch',
                                                'geodaten@sg.ch',
                                                'info@example.com',
                                                'agi@tg.ch',
                                                'mail@lisag.ch',
                                                'gis@bd.zh.ch',
                                                'info.diffusion@vd.ch',
                                                'webgis@swisstopo.ch'))
                                                else x['abstract'], axis=1)
len(data[(data['abstract'] != 'nan') &
     (data['abstract'] != data['title']) &
     (data['abstract_w_count']>4)])

c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for 

5939

In [15]:
# iterate through the dataframe
task = "Extract a list of key words comma separated without adjectives in original language from the following text: "
en_trns_task = "Translate the following list of words into a comma separated list in English: "
de_trns_task = "Übersetze die folgende Liste von Wörtern in eine durch Komma getrennte Liste auf Deutsch: "
fr_trns_task = "Traduisez la liste de mots suivante en une liste séparée par des virgules en francais: "
it_trns_task = "Traduci il seguente elenco di parole in un elenco separato da virgole in italiano: "

df_kg = pd.DataFrame()

for idx, row in tqdm(data.iterrows()):
    if idx > 100:
        continue
    trns_dict = {"ENG":en_trns_task, "DEU":de_trns_task, "ITA":it_trns_task,'FRA':fr_trns_task}
    if row['lang_3'] != 'NAN':
        del trns_dict[row['lang_3']]
    # filter rows with no abstract and use title instead
    if row['abstract'] == 'nan' or row['abstract']==row['title'] or len(row['abstract'].split(' '))<5 or row['abstract'].startswith(('??',
                                                                                                                                      'Es werden die Daten im Zeitraum',
                                                                                                                                      'Link zu Metadaten:',
                                                                                                                                      'https:',
                                                                                                                                      'geo@bs.ch',
                                                                                                                                      'info.geoportal@be.ch',
                                                                                                                                      'sit@jura.ch',
                                                                                                                                      'geodaten@sg.ch',
                                                                                                                                      'info@example.com',
                                                                                                                                      'agi@tg.ch',
                                                                                                                                      'mail@lisag.ch',
                                                                                                                                      'gis@bd.zh.ch',
                                                                                                                                      'info.diffusion@vd.ch',
                                                                                                                                      'webgis@swisstopo.ch')):
        # llama_answer = ask_llama(task, row['title'], idx)
        pass
    else:
        # generate LLM response with abstract and interpret it
        kwds = read_keyowrds(ask_llama(task, row['abstract']))

        df_kg = collect_keywords(kwds, row['lang_3'], trns_dict, df_kg)
        # for lang in [k for k in trns_dict.keys()]:
        #     llama_translation = ask_llama(trns_dict[lang], ", ".join(kwds), idx, check_response=False)
        #     translations[lang] = read_translation(llama_translation)
        # if check_length(translations, len(kwds)):
        #     df = pd.DataFrame({row['lang_3']:kwds})
        #     for lang in [k for k in trns_dict.keys()]:
        #         df[lang] = translations[lang]
        #         # data.loc[idx, 'kg_'+lang] = ','.join(translations[lang])
        #     # data.loc[idx, 'kg_'+row['lang_3']] = ','.join(kwds)
        #     df_kg = pd.concat([df_kg, df], axis=0, ignore_index=True)
        # else:
        #     print("Differing translation lengths!")

remove_rows = df_kg.applymap(lambda x: len(x.split(" "))).apply(sum, axis=1).loc[lambda x:x==4].index.tolist()
df_kg.drop(remove_rows, inplace=True)

0it [00:00, ?it/s]c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
2it [03:39, 111.17s/it]

Skipping differing translation lengths!


61it [1:12:09, 45.98s/it]

Skipping differing translation lengths!


75it [1:25:45, 56.59s/it]

...

87it [1:42:45, 84.20s/it] 

Skipping differing translation lengths!


88it [1:44:24, 88.48s/it]

Skipping differing translation lengths!


39395it [1:57:41,  5.58it/s] 


#### Clean the resulting dataframe

In [3]:
df_kg = pd.read_pickle('kg_data.pkl')
df_kg = filter_translations(df_kg)
for i, row in df_kg.iterrows():
    if len(row['ITA'].split()) < 2 and len(row['DEU'].split())<2 and len(row['ENG'].split())<2 and len(row['FRA'].split())<2:
        df_kg.drop(i, inplace=True)
df_kg.drop(columns=['NAN'], inplace=True)
df_kg['ITA'] = df_kg['ITA'].str.replace('.', '')
df_kg['DEU'] = df_kg['DEU'].str.replace('.', '')
df_kg['ENG'] = df_kg['ENG'].str.replace('.', '')
df_kg['FRA'] = df_kg['FRA'].str.replace('.', '')
to_drop = [2,12,16,21,20,23,24,30,37,44]
df_kg.drop(to_drop, inplace=True)
df_kg = df_kg.reset_index(drop=True)


to_drop = [5,6,19,5,5,90,91,178,173,110, 180, 202, 285, 382, 456,196, 299, 842]
to_drop.extend([k for k in range(8, 12)])
# to_drop.extend([k for k in range(8, 12)])
df_kg.drop(to_drop, inplace=True)

C:\Users\ma1021525\AppData\Local\Temp\ipykernel_23012\1071384662.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_kg['ITA'] = df_kg['ITA'].str.replace('.', '')
C:\Users\ma1021525\AppData\Local\Temp\ipykernel_23012\1071384662.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_kg['DEU'] = df_kg['DEU'].str.replace('.', '')
C:\Users\ma1021525\AppData\Local\Temp\ipykernel_23012\1071384662.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_kg['ENG'] = df_kg['ENG'].str.replace('.', '')
C:\Users\ma1021525\AppData\L

In [74]:
search = 'Wohnungsinventar'
col = 'title'
print(data.loc[data['title'].str.contains(search)][[col+'_de',col+'_en',col+'_it',col+'_fr']].to_markdown())

|       | title_de         | title_en          | title_it                    | title_fr                 |
|------:|:-----------------|:------------------|:----------------------------|:-------------------------|
| 19915 | Wohnungsinventar | Housing inventory | Inventario delle abitazioni | Inventaire des logements |
| 21218 | Wohnungsinventar | Housing inventory | Inventario delle abitazioni | Inventaire des logements |
| 38593 | Wohnungsinventar | Housing inventory | Inventario delle abitazioni | Inventaire des logements |


In [54]:
df_kg[df_kg['DEU'] =='Gefahrenkarte']['ITA'].unique()

array(['Carta dei pericoli', 'Carta di pericolo', 'Mappa dei pericoli',
       'Carta di pericoli', 'Grafico di pericoli'], dtype=object)

In [32]:
df_kg = pd.read_csv('kg_data.csv', delimiter=';', low_memory=False, header=0)

In [33]:
df_kg.drop(columns=['ENG_cap','ITA_cap','FRA_cap'], inplace=True)
df_kg.head()

,DEU,ENG,ITA,FRA
0,Alpkataster,alpine catalogue,catasto alpino,catalogue des alpages
1,Amphibienvorkommen,amphibian occurrences,presenza di anfibi,présence d amphibiens
2,Stand,stand,punto vendita,stand
3,Landeskarten,national maps,cartine territoriali,cartes régionales
4,Übersichtsplan,overview plan,piano di insieme,plan d ensemble


In [16]:
df_kg.to_pickle('kg_data_cleaned.pkl')

In [14]:
df_kg.loc[df_kg['DEU'] == 'Brandmeldeanlagen']

,DEU,ENG,ITA,FRA
87,Brandmeldeanlagen,fire alarm systems,sistemi di allarme antincendio,systèmes d alarme incendie


#### Generate the knowledge graph

In [7]:
df_kg = pd.read_pickle('kg_data.pkl')
# for i, row in df_kg.iterrows():
#     if len(row['ITA']) < 5 or len(row['DEU'])<5 or len(row['ENG'])<5 or len(row['FRA'])<5:
#         df_kg.drop(i, inplace=True)

In [21]:
kg = generate_knowledge_graph("GeoHarvester",'kg_data.pkl', 'knowledge_graph', load_synonyms=True)

KG: Filtering translations...
KG: Loading data in the knowledge graph...
KG: Building synonyms...


In [7]:
lang_terms = find_nodes_by_language(kg, 'italian')
[t for t in lang_terms if t in 'sistemi di allarme antincendio' ]

['sistemi di allarme antincendio', 'incendio', 'ar']

In [32]:
if [k['id'] for k in kg.v(vertex='oasi ').inc("synonym").all()['result']]:
    print('A')

In [13]:
for ref in find_nodes_by_language(kg, 'german'):
    print([k['id'] for k in kg.v(vertex=ref).inc("means").all()['result']])

['référence géographique', 'relazione geografica', 'geographical reference']
['grid format', 'formato a raggiera', 'format raster', 'formato a rete', 'raster format']
['points fixes de niveau', 'punti di riferimento per la gestione delle lagune', 'lagefixpunktes']
['possibilités de culture', 'possibilità di coltivazione.', 'farming possibilities.']
['conditions climatiques préalables', 'condizioni climatiche presupposte', 'climatic preconditions']
['geodonnées', 'geodati', 'geodetic data']
["lieu d'accident", "luogo dell'incidente", 'accident location']
['installation à feu', 'installazione di tiro', 'shooting range']
["site d'exploitation", 'posizione operativa', 'operating location']
['déchalandage ancien', 'deposito di rifiuti', 'landfill']
['ancienne pollution', 'rischio ambientale', 'hazardous waste site']
['installations militaires', 'installazioni militari', 'military installations']
['espèces animales.', 'specie animali', 'species types']
['région cantonale', 'distretto cantona

In [22]:
find_translation(kg, 'Brandmeldeanlagen', 'german')

['systèmes d alarme incendie',
 'sistemi di allarme antincendio',
 "systèmes d'alarmes incendie",
 "sistemi d'allarme antincendio",
 'fire alarm systems',
 'Brandmeldeanlagen']

In [45]:
kg.v("carta dei pericoli").tag("from").out("means").tag("to").view("synonym").render()